In [ ]:
### Initial design of drawing a coplanar waveguide resonator 


# automatic reloading of modules when they change

%load_ext autoreload
%autoreload 2


In [ ]:
# Import statements

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr

from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

In [ ]:
# Instantiate QDesign class and initialise user interface

design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'
design.chips.main.size['size_z'] = '-500um'
design.chips.main.size['sample_holder_bottom'] = '500um'
design.chips.main.size['sample_holder_top'] = '2000um'


# Set hfss as renderer for design
hfss = design.renderers.hfss

In [ ]:
# Allow overwriting
gui = MetalGUI(design)
design.overwrite_enabled = True

In [ ]:
#Add hanger of capacitively coupled transmission lines
TQ1 = CoupledLineTee(design, 'TQ1', options=dict(pos_x='0mm',
                     pos_y='0.8mm',
                     coupling_length='100um'))

gui.rebuild()
gui.autoscale()

In [ ]:
stg = ShortToGround(design, 'short_to_ground', options=dict(pos_x='0um', pos_y='0.5mm',
orientation='270')) 


In [ ]:
# Create open pin for resonator

otg1 = OpenToGround(design, 'open_to_ground1', options=dict(pos_x='0um', pos_y='-0.95mm',
orientation='180'))


# Jogs to allow bending of open end of CPW
# So far does not render in hfss
from collections import OrderedDict
jogs = OrderedDict()
jogs[0] = ["L", '125um']
jogs[1] = ["R", '100um']
#jogs[2] = ["L", '100um']

jogs2 = OrderedDict()
jogs2[0] = ["R", '100um']
jogs2[1] = ["L", '110um' ]

# Create meandered resonator between shorted and opened pin
rt_meander = RouteMeander(design, 'readout', Dict(
        total_length='4.18 mm',
        hfss_wire_bonds = True,
        fillet='70 um',
        #fillet='80 um',
        lead = Dict(
        start_straight='0.1mm',
        end_straight='0mm'),
        #start_jogged_extensions=jogs2,
        #end_jogged_extension=jogs),
        meander=Dict(
        #spacing = '200um'),
        spacing = '150um'),
        pin_inputs=Dict(
            #start_pin=Dict(component='TQ1', pin='second_end'),
            start_pin=Dict(component='short_to_ground', pin='short'),
            end_pin=Dict(component='open_to_ground1', pin='open')), ))

gui.rebuild()
gui.autoscale()

In [ ]:
#rt_meander.options
design.chips

In [ ]:
#Setup the launchpad location and orientation.
#launch_options1 = dict(pos_x='-500um', pos_y='0.8mm', orientation='0')

#lp = LaunchpadWirebond(design, 'P4_Q', options = launch_options1)

otg2 = OpenToGround(design, 'open_to_ground2', options=dict(pos_x='-1.3mm', pos_y='0.8mm',
orientation='0'))



#Setup the OpenToGround location and orientation.
#launch_options2 = dict(pos_x='0.5mm',  pos_y='0.8mm', orientation='180')

#lp2 = LaunchpadWirebond(design, 'P5_Q', options = launch_options2)


otg3 = OpenToGround(design, 'open_to_ground3', options=dict(pos_x='1.3mm', pos_y='0.8mm',
orientation='180'))

#After the two QComponents are added to design, connect them with a RouteMeander.
meander_options_left = Dict(
        hfss_wire_bonds = True,
        pin_inputs=Dict(
            start_pin=Dict(component='TQ1', pin='prime_start'),
            end_pin=Dict(component=otg2.name, pin='open')), )

meander_options_right = Dict(
        hfss_wire_bonds = True,
        pin_inputs=Dict(
            start_pin=Dict(component='TQ1', pin='prime_end'),
            end_pin=Dict(component=otg3.name, pin='open')), )


meander_left = RouteStraight(design, 'cpw_left',options=meander_options_left)
meander_right = RouteStraight(design, 'cpw_right',options=meander_options_right)

gui.rebuild()
gui.autoscale()

In [ ]:
lp.pin_names

In [ ]:
hfss.open_ansys(path = 'C:\Program Files\AnsysEM\AnsysEM19.3\Win64')

In [ ]:
hfss.connect_ansys()

In [ ]:
##############################################
# BELOW cells are for drivenmodal simulation #
##############################################

In [ ]:
hfss.activate_drivenmodal_design("S-parameter estimation")

In [ ]:
hfss.clean_active_design()

In [ ]:
hfss.add_mesh()

In [ ]:
hfss.render_design([],
                   [],
                  [('cpw_right', 'end', 50), ('cpw_left', 'end', 50)],
                  #[(lp.name, 'tie', 50), (lp2.name, 'tie', 50)],
                  [],
                  [],
                  box_plus_buffer=False)

In [ ]:
design._chips['main']['size']['size_x'] = '2.6mm'
design._chips['main']['size']['size_y'] = '2.6mm'

In [ ]:
hfss.default_options

In [ ]:
hfss.add_sweep(setup_name="Setup",
               name="Sweep",
               start_ghz=5.0,
               stop_ghz=9.0,
               count=20001,
               type="Interpolating")

In [ ]:
hfss.analyze_sweep('Sweep', 'Setup')

In [ ]:
hfss.plot_params(['S11', 'S21'])

In [ ]:
#################################
# END OF DRIVENMODAL SIMULATION #
#################################
############################################

In [ ]:
############################################
# BELOW CELLS ARE FOR EIGENMODE SIMULATION #
############################################



In [ ]:
hfss.activate_eigenmode_design("Readout")

In [ ]:
design._chips['main']['size']['size_x'] = '6mm'
design._chips['main']['size']['size_y'] = '5.5mm'

In [ ]:
hfss.render_design([stg.name, otg1.name, 'readout'],[])

In [ ]:
setup = hfss.pinfo.setup
setup.passes = 20
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
pinfo.design.set_variable('Lj', '10 nH')
pinfo.design.set_variable('Cj', '0 fF')
setup.analyze()

hfss.plot_convergences()

In [ ]:
hfss.disconnect_ansys()
